In [34]:
import numpy as np
import ConformaQuantile as CQ
import importlib
importlib.reload(CQ)

<module 'ConformaQuantile' from '/Users/wangyanbo/conformal/ConformaQuantile.py'>

In [37]:
quantile = 0.8
n_samples = 3000
n_X1 = 20
n_X2 = 2
n_X3 = 2
np.random.seed(0)

X1 = np.random.normal(3, 10, (n_samples, n_X1))
X2 = np.random.normal(0.4, 1, (n_samples, n_X2))
X3 = np.random.normal(0.9, 1, (n_samples, n_X3))

coefficients = np.random.normal(1, 0.5, n_X1 + n_X2)
X = np.hstack((X1, X2, X3))
noise = np.random.normal(0, 1, n_samples)

X_true = X[:, :(n_X1 + n_X2)]
X_observed = np.hstack((X1, X3))
Y = np.dot(X_true, coefficients)

train_ratio = 0.6
validation_ratio = 0.2
test_ratio = 0.2

In [38]:
Y0 = Y + noise
Y2 = -Y**(-1) + noise
# Y3 = 1 / ( 1 + np.exp(-Y)) + noise
Y3 = np.sin(Y) + noise
Y4 = Y**(2) + noise

In [39]:
Y5 = np.zeros(n_samples)  
count = np.zeros(n_samples)

for i in range(n_samples):
    if X1[i, 0] > 3 and X1[i, 1] > 3:
        Y5[i] = Y0[i]
    elif X1[i, 0] > 3 and X1[i, 1] <= 3:
        Y5[i] = Y2[i]
    elif X1[i, 0] <= 3 and X1[i, 1] > 3:
        Y5[i] = Y3[i]
    else:
        Y5[i] = Y4[i]
loss = {}
models = ['linear', 'quantile', 'lasso', 'ridge', 'random_forest', 'glm', 'neural_network', 'ko']
for model in models:
    loss_unadjusted, loss_adjusted = CQ.perform_regression_analysis(X_observed, Y5, train_ratio, test_ratio, validation_ratio, quantile, model_type=model)
    # 将结果存储在字典中
    loss[model] = {'loss_unadjusted': loss_unadjusted, 'loss_adjusted': loss_adjusted}


min_loss_model = min(loss, key=lambda x: loss[x]['loss_adjusted'])
max_loss_model = max(loss, key=lambda x: loss[x]['loss_adjusted'])

print(f"\n拥有最小调整损失的模型：{min_loss_model}")
print(f"\n拥有最大调整损失的模型：{max_loss_model}")

linear loss unadjusted 726.3851637443138 loss_adjusted 509.30470490993116
quantile loss unadjusted 603.3004770229385 loss_adjusted 490.14900473945823
lasso loss unadjusted 726.3735052413232 loss_adjusted 509.289226704787
ridge loss unadjusted 726.3828810416488 loss_adjusted 509.30264018803894
random_forest loss unadjusted 598.344393366171 loss_adjusted 230.51191005254756
glm loss unadjusted 726.3851637443136 loss_adjusted 509.30470490993116
neural_network loss unadjusted 531.3612518796092 loss_adjusted 502.3567744532573


/Users/wangyanbo/conformal/ConformaQuantile.py:56: RuntimeWarning: invalid value encountered in double_scalars
  if indicator_sum / sum(kernel_weights) < quantile:


ko loss unadjusted 602.126419151474 loss_adjusted 518.6496518555065

拥有最小调整损失的模型：random_forest

拥有最大调整损失的模型：ko
